# [Database Secret Engine](https://developer.hashicorp.com/vault/tutorials/db-credentials/database-secrets)

In [ ]:
%env WORKDIR=/tmp/vault            

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')

## Install PostgreSQL

In [ ]:
%%bash

export DOCKER_CONFIG=$HOME/.config/containers

# Add Helm repository by Bitnami
helm repo add bitnami https://charts.bitnami.com/bitnami

# Update Helm index charts
helm repo update
kubectl apply -f local-pv.yaml
kubectl apply -f pv-claim.yaml

helm install postgresql-dev -f postgres.yaml oci://registry-1.docker.io/bitnamicharts/postgresql -n vault --set volumePermissions.enabled=true

In [ ]:
! kubectl get events -n vault

Create role that will be used by Vault

In [ ]:
! kubectl exec -it postgresql-dev-0 -n vault -- sh -c 'export PGPASSWORD=StrongPassword; psql --host 127.0.0.1 -U postgres -c "CREATE ROLE \"ro\" NOINHERIT;"'
! kubectl exec -it postgresql-dev-0 -n vault -- sh -c 'export PGPASSWORD=StrongPassword; psql --host 127.0.0.1 -U postgres -c "GRANT SELECT ON ALL TABLES IN SCHEMA public TO \"ro\";"'

## Enable the database secret engine

In [ ]:
%%bash
export POSTGRES_URL=postgresql-dev.vault.svc.cluster.local
# Enable engine
vault secrets enable database

# Configure the database secrets engine with the connection credentials for the Postgres database.
vault write database/config/postgresql \
     plugin_name=postgresql-database-plugin \
     connection_url="postgresql://{{username}}:{{password}}@$POSTGRES_URL/postgres?sslmode=disable" \
     allowed_roles=* \
     username="postgres" \
     password="StrongPassword"

## Create a role in Vault

In [ ]:
%%bash
cat > ${WORKDIR}/readonly.sql <<EOF
CREATE ROLE "{{name}}" WITH LOGIN PASSWORD '{{password}}' VALID UNTIL '{{expiration}}' INHERIT;
GRANT ro TO "{{name}}";
EOF

In [ ]:
%%bash
vault write database/roles/readonly \
      db_name=postgresql \
      creation_statements=@${WORKDIR}/readonly.sql \
      default_ttl=60s \
      max_ttl=120s

## Request Credentials

In [ ]:
! vault read -format=json database/creds/readonly 

In [ ]:
! vault lease revoke -prefix database/creds/readonly

## Rotate Root password

In [ ]:
! vault write -force database/rotate-root/postgresql

## Clean up

In [ ]:
%%bash

vault secrets disable database
helm uninstall postgresql-dev -n vault
kubectl delete pvc postgresql-data-claim -n vault
kubectl delete pv postgresql-data -n vault